# Web Scraping - Regente

In [9]:
import time

import undetected_chromedriver as uc

from selenium.webdriver.remote.webdriver import By
import selenium.webdriver.support.expected_conditions as EC  # noqa
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.action_chains import ActionChains


In [39]:
def set_viewport_size(driver, width, height):
    window_size = driver.execute_script("""
        return [window.outerWidth - window.innerWidth + arguments[0],
          window.outerHeight - window.innerHeight + arguments[1]];
        """, width, height)
    driver.set_window_size(*window_size)

In [2]:
driver = uc.Chrome()
driver.get("https://superregente.instabuy.com.br/departamentos")

In [3]:
categoriesLinks = []

categoriesLiList = driver.find_elements(By.XPATH, "/html/body/app-root/ng-component/div/div[2]/div/div/*") 

for li in categoriesLiList:
    link = li.find_element(By.CSS_SELECTOR, "li.title > a").get_attribute("href")
    categoriesLinks.append(link)
    
categoriesLinks

['https://superregente.instabuy.com.br/cat/Alimentos-Basicos',
 'https://superregente.instabuy.com.br/cat/Frios-e-Laticinios',
 'https://superregente.instabuy.com.br/cat/Mercearia',
 'https://superregente.instabuy.com.br/cat/Acougue-Aves-Peixaria',
 'https://superregente.instabuy.com.br/cat/Frutas-Legumes-e-Verduras',
 'https://superregente.instabuy.com.br/cat/Bebidas-Alcoolicas',
 'https://superregente.instabuy.com.br/cat/Bebidas-Nao-Alcoolicas',
 'https://superregente.instabuy.com.br/cat/Biscoitos-e-Aperitivos',
 'https://superregente.instabuy.com.br/cat/Condimentos-e-Molhos',
 'https://superregente.instabuy.com.br/cat/Doces',
 'https://superregente.instabuy.com.br/cat/Congelados',
 'https://superregente.instabuy.com.br/cat/Higiene-e-Perfumaria',
 'https://superregente.instabuy.com.br/cat/Limpeza',
 'https://superregente.instabuy.com.br/cat/Padaria',
 'https://superregente.instabuy.com.br/cat/Outros',
 'https://superregente.instabuy.com.br/cat/Bebes',
 'https://superregente.instabuy.

In [30]:
def getProximoButton():
	try:
		pagination_list = driver.find_elements(By.XPATH, "/html/body/app-root/sub-category/div[2]/div/main/ib-pagination/div/div/*")
		if (len(pagination_list) == 0): return None

		li = pagination_list[-1]
		span = li.find_element(By.TAG_NAME, "span")
		span_classes = span.get_attribute("class").split(" ")
		return span if all([clss != "disabled" for clss in span_classes]) else None
	except NoSuchElementException:
		return None

getProximoButton()

In [38]:
import re
product_list = []

baseLink = "https://superregente.instabuy.com.br/cat/Mercearia"
proximoButton = True

# Para buscar também as diferentes categorias:

# for link in categoriesLinks:
#   driver.get(link)
#   time.sleep(2)

sub_categories_list = list(map(lambda cat : cat.find_element(By.TAG_NAME, "a").get_attribute("href"), 
															 driver.find_elements(By.XPATH, "/html/body/app-root/category/div[2]/div/div[2]/div/ul/*")))

for sub_category in sub_categories_list:
	driver.get(sub_category)

	proximoButton = True

	while(proximoButton != None):

		product_elements_list = WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, "/html/body/app-root/sub-category/div[2]/div/main/div[2]/div/*")))

		for product in product_elements_list:
			name = product.find_element(By.CSS_SELECTOR, "span.ib-single-item-info-name").text
			link = product.find_element(By.CSS_SELECTOR, "a.ib-single-item-info").get_attribute("href")
			image = product.find_element(By.CSS_SELECTOR, "div.ib-single-item-image > a > img").get_attribute("src")
			price = product.find_element(By.CSS_SELECTOR, "span.ib-single-item-info-price").text

			price = re.sub("R\$([0-9]+),([0-9]+)", "\g<1>.\g<2>", price)

			product_list.append({"name": name, "price": price, "link": link, "image": image, "store": "Regente Supermercado"})

		proximoButton = getProximoButton()

		if (proximoButton != None):
			print(proximoButton.get_attribute("class"))
			proximoButton.click()

product_list

[{'name': 'Chocolate em Pó Solúvel 50% Cacau NESTLÉ Dois Frades Caixa 200g',
  'price': '17.89',
  'link': 'https://superregente.instabuy.com.br/p/Chocolate-em-Po-Soluvel-50-Cacau-Nestle-Dois-Frades-Caixa-200g',
  'image': 'https://ibassets.com.br/ib.item.image.medium/m-bbace77142af4067a407cb7224fce3bd.jpeg',
  'store': 'Regente Supermercado'},
 {'name': 'Nescau Nestle 550G',
  'price': '14.29',
  'link': 'https://superregente.instabuy.com.br/p/Nescau-Nestle-550G',
  'image': 'https://ibassets.com.br/ib.item.image.medium/m-ad744f41680a4c3fa65388e04eeabf37.jpeg',
  'store': 'Regente Supermercado'},
 {'name': 'Cacau Em Pó 100% Zaelei 200G',
  'price': '17.99',
  'link': 'https://superregente.instabuy.com.br/p/Cacau-Po-100-Zaelei-200G',
  'image': 'https://ibassets.com.br/ib.item.image.medium/m-4c0c01116ef94af4b539c883c9bd4fc1.png',
  'store': 'Regente Supermercado'},
 {'name': 'Achocolatado em Pó Powerlate SÃO BRAZ 300g',
  'price': '6.99',
  'link': 'https://superregente.instabuy.com.br

In [40]:
if (len(product_list) > 0):
    import csv
    import json
    from datetime import datetime
    
    now = datetime.now().strftime("%Y%m%dT%H:%M:%S")

    csvPath = f"data/regente_{now}.csv"
    jsonPath= f"data/regente_{now}.json"


    fieldnames = product_list[0].keys()
    with open(csvPath, 'w', encoding='UTF8', newline='') as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(product_list)

    print(f"Arquivo {csvPath} criado com sucesso!")
    
    with open(jsonPath, 'w', encoding='utf-8', newline='') as json_file:
        json.dump(product_list, json_file, ensure_ascii=False)

    print(f"arquivo {jsonPath} criado com sucesso!")
    
else:
    raise AssertionError("A lista de produtos extraídos está vazia")


Arquivo data/regente_20230910T01:05:46.csv criado com sucesso!
arquivo data/regente_20230910T01:05:46.json criado com sucesso!
